In [ ]:
from fastplotlib import Plot, GridPlot
import numpy as np
import random

In [ ]:
# Static Buffer w/ Full Buffer Replacement

## Set grid_plot attributes
grid_shape = (2, 3)

controllers = [
    [0, 3, 1],  
    [2, 2, 3]
]

names = [
    ["subplot0", "subplot1", "subplot2"],
    ["subplot3", "subplot4", "subplot5"]
]

# Create the grid plot
grid_plot = GridPlot(
    shape=grid_shape,
    controllers=controllers,
    names=names
)

image_graphics = list()

# Make a random line for each subplot
for subplot in grid_plot:
    xs = np.linspace(0, 1000, 1000)
    ys = random.sample(range(0,1000), 1000)
    data = np.dstack([xs, ys])[0]
    ig = subplot.add_line(data=data, cmap='jet', name="image")
    image_graphics.append(ig)
    

# Define a function to update the image graphics 
# with new randomly generated data
def set_random_frame():
    for ig in image_graphics:
        ys = random.sample(range(0,1000), 1000)
        new_data = np.dstack([xs, ys])[0]
        ig.data = new_data
# add the animation
grid_plot.add_animations(set_random_frame)
grid_plot.show()

In [ ]:
# Static Buffer w/ Partial Replacement (Rolling Buffer)
## Simple Indexing

# Set window size
window_length = 10

## Set grid_plot attributes
grid_shape = (2, 3)

controllers = [
    [0, 3, 1],  
    [2, 2, 3]
]

names = [
    ["subplot0", "subplot1", "subplot2"],
    ["subplot3", "subplot4", "subplot5"]
]

# Create the grid plot
grid_plot = GridPlot(
    shape=grid_shape,
    controllers=controllers,
    names=names
)

image_graphics = list()

# Make a random line for each subplot
for subplot in grid_plot:
    xs = np.linspace(0, 1000, 1000)
    ys = random.sample(range(0,1000), 1000)
    data = np.dstack([xs, ys])[0]
    ig = subplot.add_line(data=data, cmap='jet', name="image")
    image_graphics.append(ig)

# Define a function to update the image graphics 
# with new randomly generated data
def set_random_frame():
    for ig in image_graphics:
        ys = random.sample(range(0,1000), window_length)
        new_data_points = np.dstack([xs[-window_length:], ys])[0]
        data[:-window_length] = data[window_length:] - [window_length,0]
        data[-window_length:] = new_data_points
        ig.data = data
# add the animation
grid_plot.add_animations(set_random_frame)
grid_plot.show()

In [ ]:
import multiprocessing
from multiprocessing.shared_memory import SharedMemory
import time
import numpy as np
import random
from fastplotlib import Plot, GridPlot

def generate_data(generate_ef, acquire_ef, mutex):
    for i in range(10):
        # Wait for event flag to be set
        generate_ef.wait()
        # Load shared memory object
        shm = SharedMemory(sn)
        np_array = np.ndarray(shape=shape, dtype=dtype, buffer=shm.buf)
        # Acquire/release mutex, prevent parallel access to object
        mutex.acquire()

        for i,ig in enumerate(image_graphics):
            ys = np.array(random.sample(range(0,1000), window_length))
            np_array[i, :-window_length] = np_array[i, window_length:]
            np_array[i, -window_length:] = ys
        mutex.release()
        # Clear ef, set next ef
        generate_ef.clear()
        acquire_ef.set()

def _visualize_data():
    # for i in range(100):
#     acquire_ef.wait()
    print("successful entry")
    shm = SharedMemory(sn)
    np_array = np.ndarray(shape=shape, dtype=dtype, buffer=shm.buf)
#     mutex.acquire()
    for i,ig in enumerate(image_graphics):
        data = np.dstack([xs,np_array[i]])[0]
        ig.data = data
        
    grid_plot.show()
#     mutex.release()
#     acquire_ef.clear()
#     generate_ef.set()

def visualize_data(generate_ef, acquire_ef, mutex):
    for i in range(10):
        acquire_ef.wait()
        print("start animation")
        shm = SharedMemory(sn)
        np_array = np.ndarray(shape=shape, dtype=dtype, buffer=shm.buf)
        mutex.acquire()
        for i,ig in enumerate(image_graphics):
            data = np.dstack([xs,np_array[i]])[0]
            ig.data = data
        #grid_plot.add_animations(_visualize_data)
        mutex.release()
        grid_plot.show()
        acquire_ef.clear()
        generate_ef.set()
        
def create_shared_block(dim, grid_plot):
    global xs
    xs = np.linspace(0,dim,dim)
    ys = np.ones(shape=(6,dim), dtype=np.int64)
    image_graphics = list()
    for i, subplot in enumerate(grid_plot):
        ys[i] =  random.sample(range(0,dim), dim)
        data = np.dstack([xs, ys[i]])[0]
        ig = subplot.add_line(data=data, cmap='jet', name="image")
        image_graphics.append(ig)

    shm = SharedMemory(create=True, size=ys.nbytes)
    # # Now create a NumPy array backed by shared memory
    np_array = np.ndarray(ys.shape, dtype=np.int64, buffer=shm.buf)
    np_array[:] = ys[:]  # Copy the original data into shared memory
    return shm, xs, np_array, image_graphics

def create_event_flags():
    generate_ef = multiprocessing.Event()
    acquire_ef = multiprocessing.Event()
    return generate_ef, acquire_ef

def create_gridplot():
    ## Set grid_plot attributes
    grid_shape = (2, 3)

    controllers = [
        [0, 3, 1],  
        [2, 2, 3]
    ]

    names = [
        ["subplot0", "subplot1", "subplot2"],
        ["subplot3", "subplot4", "subplot5"]
    ]

    # Create the grid plot
    grid_plot = GridPlot(
        shape=grid_shape,
        controllers=controllers,
        names=names
    )
    
    return grid_plot

if __name__ == '__main__':
    global window_length, shape, dtype, sn, grid_plot, image_graphics
    window_length=10
    # Initialize Grid Plot
    grid_plot = create_gridplot()
    # Create Event Flags
    generate_ef, acquire_ef = create_event_flags()
    # Initialize shared memory block
    shm, xs, np_array, image_graphics = create_shared_block(1000, grid_plot)
    shape, dtype = np_array.shape, np_array.dtype
    sn = shm.name
    # Initialize mutex
    mutex = multiprocessing.Lock()
    # Define processes
    p1 = multiprocessing.Process(name='generate', 
                                 target=generate_data,
                                 args=(generate_ef, acquire_ef, mutex))
    p2 = multiprocessing.Process(name='visualize', 
                                 target=visualize_data, 
                                 args=(generate_ef, acquire_ef, mutex))
#     # Start Processes
    p1.start()
    p2.start()
    
#     # Release generate_ef
    generate_ef.set()
    
    p1.join()
    p2.join()
    

RFBOutputContext()

fastplotlib.GridPlot @ 0x7f68ac11ca60



Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_455777/2972543870.py", line 55, in stream_data
    ys_shared[:]+=1
numpy.core._exceptions._UFuncNoLoopError: ufunc 'add' did not contain a loop with signature matching types (dtype('V8'), dtype('int64')) -> None
